Este trabalho tem como objetivo a construção de uma pipeline completa de supervisão fraca e/ou detecção de erros de anotação aplicada ao conjunto de dados Endoscopic Bladder Tissue Classification Dataset, disponível publicamente na plataforma Kaggle por meio do link: https://www.kaggle.com/datasets/aryashah2k/endoscopic-bladder-tissue-classification-dataset/data. 
Trata-se de um conjunto de dados de imagens endoscópicas da bexiga, criado com o objetivo de auxiliar na classificação de tecidos vesicais em diferentes categorias, visando o suporte ao diagnóstico médico de câncer de bexiga.
As imagens foram capturadas por endoscopia durante procedimentos clínicos reais e estão organizadas nas classes de tecido:
HGC (High-Grade Cancer), LGC (Low-Grade Cancer), NST (Neoplastic Suspected Tissue) e NTL (Normal Tissue Lesion).
Neste contexto, a proposta visa aplicar técnicas de aprendizado com supervisão fraca, como a detecção de rótulos ruidosos (anotações incorretas), a fim de identificar possíveis inconsistências nas anotações originais. Essa abordagem tem como finalidade melhorar a qualidade dos dados, o que pode refletir diretamente em uma melhor performance dos modelos de classificação treinados sobre esse conjunto.
O dataset é composto por 1.754 imagens, distribuídas entre subconjuntos de treino, validação e teste, e organizadas nas quatro classes mencionadas.



In [1]:
pip install cleanlab scikit-learn opencv-python-headless


   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
    --------------------------------------- 0.5/39.4 MB 1.9 MB/s eta 0:00:21
   - -------------------------------------- 1.8/39.4 MB 4.4 MB/s eta 0:00:09
   --- ------------------------------------ 3.1/39.4 MB 5.0 MB/s eta 0:00:08
   ---- ----------------------------------- 4.5/39.4 MB 5.5 MB/s eta 0:00:07
   ----- ---------------------------------- 5.8/39.4 MB 5.6 MB/s eta 0:00:07
   ------- -------------------------------- 7.1/39.4 MB 5.7 MB/s eta 0:00:06
   -------- ------------------------------- 8.4/39.4 MB 5.7 MB/s eta 0:00:06
   --------- ------------------------------ 9.7/39.4 MB 5.8 MB/s eta 0:00:06
   ----------- ---------------------------- 11.0/39.4 MB 5.7 MB/s eta 0:00:05
   ------------ --------------------------- 12.3/39.4 MB 5.8 MB/s eta 0:00:05
   ------------- -------------------------- 13.6/39.4 MB 5.9 MB/s eta 0:00:05
   --------------- ------------------------ 14.9/39.4 MB 5.9 MB/s eta 0:00:05
  

Foi realizada uma detecção automática de rótulos possivelmente errados com a biblioteca Cleanlab.
Etapas da pipeline: 

1. Leitura do dataset
   - O arquivo `annotations.csv` é carregado, contendo os caminhos das imagens, o tipo de tecido anotado e a indicação se a amostra pertence ao conjunto de treino ou teste.
   - Cada imagem recebe um rótulo numérico correspondente à sua classe (`HGC`, `LGC`, `NST` ou `NTL`).

2. Extração de características das imagens
   - As imagens são processadas com OpenCV:
     - Redimensionadas para 64×64 pixels.
     - Convertidas para escala de cinza.
     - Extraído um histograma de intensidade com 128 bins (features).
   - O vetor resultante é normalizado para cada imagem.

3. Validação cruzada (out-of-sample predictions)
   - Uma validação cruzada estratificada de 5 folds é aplicada.
   - Para cada fold, é treinado um modelo `RandomForestClassifier` e obtidas as probabilidades preditas para as amostras do fold de validação.
   - Esse processo gera previsões realistas, pois cada imagem foi avaliada por um modelo que não a viu durante o treinamento.

4. Aplicação do Cleanlab
   - Com as predições de probabilidade, o Cleanlab identifica exemplos com baixa autoconfiança (probabilidade do rótulo anotado ser menor do que a de outro rótulo).
   - O método `find_label_issues` retorna os índices dos exemplos com maior chance de estarem anotados incorretamente.

5. Exibição de rótulos suspeitos
   - Os 10 exemplos mais suspeitos são exibidos com:
     - Caminho da imagem.
     - Rótulo original.
     - Classe mais provável segundo o modelo.

In [4]:
import pandas as pd
import os
import numpy as np
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from cleanlab.filter import find_label_issues

N_SPLITS = 5  # número de folds para validação cruzada
DATA_DIR = 'C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue'
ANNOTATIONS_FILE = os.path.join(DATA_DIR, "annotations.csv")
IMAGE_CLASSES = ["HGC", "LGC", "NST", "NTL"]
class_to_id = {name: i for i, name in enumerate(IMAGE_CLASSES)}
id_to_class = {i: name for i, name in enumerate(IMAGE_CLASSES)}

# Carrega anotações
df = pd.read_csv(ANNOTATIONS_FILE)
df['image_path'] = df.apply(lambda row: os.path.join(DATA_DIR, row['tissue type'], row['HLY']), axis=1)
df['label'] = df['tissue type'].map(class_to_id)
df_train = df[df['sub_dataset'] == 'train'].reset_index(drop=True)

# Função de extração de features com OpenCV 
def extract_features(img_path):
    try:
        img = cv2.imread(img_path)
        if img is None:
            return np.zeros(128)
        img = cv2.resize(img, (64, 64))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray], [0], None, [128], [0, 256]).flatten()
        return hist / (hist.sum() + 1e-7)
    except:
        return np.zeros(128)

print("Extraindo features das imagens...")
X = np.array([extract_features(p) for p in df_train['image_path']])
y = df_train['label'].values

# Validação cruzada para gerar predições out-of-sample
print("Realizando validação cruzada...")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
pred_probs = np.zeros((len(y), len(IMAGE_CLASSES)))

for train_idx, test_idx in skf.split(X, y):
    X_train, X_val = X[train_idx], X[test_idx]
    y_train = y[train_idx]
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    pred_probs[test_idx] = model.predict_proba(X_val)

# Detecção de rótulos suspeitos
print("Executando Cleanlab com predições reais...")
label_issues = find_label_issues(
    labels=y,
    pred_probs=pred_probs,
    return_indices_ranked_by="self_confidence"
)

print(f"Cleanlab encontrou {len(label_issues)} rótulos potencialmente errados.")

# Mostrar os 10 principais
print("\nTop 10 rótulos suspeitos:")
for idx in label_issues[:10]:
    row = df_train.iloc[idx]
    original_label = id_to_class[row['label']]
    predicted_label_id = np.argmax(pred_probs[idx])
    predicted_label_name = id_to_class[predicted_label_id]
    print(f"Imagem: {row['image_path']}")
    print(f"Rótulo original: {original_label}")
    print(f"Predição mais provável: {predicted_label_name}")
    print("-" * 30)

Extraindo features das imagens...
Realizando validação cruzada...
Executando Cleanlab com predições reais...
Cleanlab encontrou 64 rótulos potencialmente errados.

Top 10 rótulos suspeitos:
Imagem: C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue\NST\case_013_pt_001_frame_0087.png
Rótulo original: NST
Predição mais provável: HGC
------------------------------
Imagem: C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue\NST\case_010_pt_002_frame_0051.png
Rótulo original: NST
Predição mais provável: HGC
------------------------------
Imagem: C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue\NTL\case_022_pt_002_frame_0037.png
Rótulo original: NTL
Predição mais provável: LGC
------------------------------
Imagem: C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue\NST\case_017_pt_004_frame_0224.png
Rótulo original: NST
Predição mais provável: LGC
------------------------------
Imagem: C:/Users/su_20/Downloads/archive/EndoscopicBladderTissue\NST\case_009_pt_001_frame

Após identificar rótulos potencialmente errados com o Cleanlab, nesta etapa realizou-se uma comparação entre o desempenho dos modelos: modelo treinado com os rótulos originais e o modelo treinado com rótulos corrigidos (retagging).
Etapas da avaliação:

1. Correção de rótulos (retagging automatizado)
   - Os rótulos suspeitos (detectados pelo Cleanlab) são substituídos pela classe com maior probabilidade predita pelo modelo.
   - Isso cria um vetor `y_corrected` com rótulos ajustados.

2. Divisão dos dados
   - Os dados são divididos em 70% para treinamento e 30% para teste, mantendo a proporção entre as classes.
   - São criados dois conjuntos de rótulos de treino:
     - `y_train_original`: com rótulos originais.
     - `y_train_corrected`: com rótulos corrigidos nas posições detectadas.

3. Treinamento e predição
   - Dois modelos `RandomForestClassifier` são treinados:
     - Um com os rótulos originais.
     - Outro com os rótulos corrigidos.
   - Ambos são testados no mesmo conjunto de teste para garantir uma comparação justa.

4. Avaliação do desempenho
   - São calculadas as seguintes métricas para os dois modelos:
     - Acurácia
     - MCC (Matthews Correlation Coefficient)
     - Relatório de classificação, contendo:
       - *Precision*
       - *Recall*
       - *F1-score* para cada classe.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, matthews_corrcoef, accuracy_score

# Rótulos antes da correção
y_original = y.copy()

# Rótulos corrigidos: apenas onde há problemas detectados
y_corrected = y.copy()
for idx in label_issues:
    y_corrected[idx] = np.argmax(pred_probs[idx])  # troca pelo rótulo mais provável

# Separar conjunto de teste (30%) para avaliação final
X_train, X_test, y_train_original, y_test = train_test_split(X, y_original, test_size=0.3, random_state=42, stratify=y_original)
_, _, y_train_corrected, _ = train_test_split(X, y_corrected, test_size=0.3, random_state=42, stratify=y_original)

# Treinar modelo com dados originais
clf_original = RandomForestClassifier(n_estimators=100, random_state=42)
clf_original.fit(X_train, y_train_original)
y_pred_original = clf_original.predict(X_test)

# Treinar modelo com dados retaggeados
clf_corrected = RandomForestClassifier(n_estimators=100, random_state=42)
clf_corrected.fit(X_train, y_train_corrected)
y_pred_corrected = clf_corrected.predict(X_test)

# Avaliação 
print("\n Desempenho com rótulos originais:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_original):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_original):.4f}")
print("Relatório de Classificação:\n", classification_report(y_test, y_pred_original, target_names=IMAGE_CLASSES))

print("\n Desempenho com rótulos corrigidos (retagging):")
print(f"Acurácia: {accuracy_score(y_test, y_pred_corrected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_corrected):.4f}")
print("Relatório de Classificação:\n", classification_report(y_test, y_pred_corrected, target_names=IMAGE_CLASSES))



 Desempenho com rótulos originais:
Acurácia: 0.8386
MCC: 0.7663
Relatório de Classificação:
               precision    recall  f1-score   support

         HGC       0.80      0.82      0.81        93
         LGC       0.79      0.90      0.84       145
         NST       0.95      0.93      0.94       114
         NTL       0.83      0.19      0.31        26

    accuracy                           0.84       378
   macro avg       0.84      0.71      0.72       378
weighted avg       0.84      0.84      0.83       378


 Desempenho com rótulos corrigidos (retagging):
Acurácia: 0.8175
MCC: 0.7349
Relatório de Classificação:
               precision    recall  f1-score   support

         HGC       0.78      0.78      0.78        93
         LGC       0.76      0.87      0.81       145
         NST       0.93      0.92      0.93       114
         NTL       0.83      0.19      0.31        26

    accuracy                           0.82       378
   macro avg       0.83      0.69     

Com base na análise das métricas, observa-se que o processo de retagging não resultou em uma melhoria de desempenho. Tanto a acurácia (que passou de 83,86% para 81,75%) quanto o coeficiente de correlação de Matthews (MCC), que caiu de 0,7663 para 0,7349, apresentaram uma leve redução após a correção automática dos rótulos.

Em contextos sensíveis como o da área médica, essa queda sugere que o retagging automatizado deve ser encarado com cautela. A revisão manual dos rótulos suspeitos identificados por ferramentas como o Cleanlab ainda se mostra essencial, de modo a verificar se as sugestões de correção realmente fazem sentido do ponto de vista clínico e diagnóstico. A triagem automatizada é útil para destacar possíveis inconsistências, mas não substitui o julgamento especializado de profissionais da saúde.

Diversos fatores podem ter contribuído para os resultados observados. O primeiro é o número reduzido de amostras do conjunto de dados — apenas 1.754 imagens — o que limita a capacidade do modelo de aprender padrões de forma robusta e compromete a estabilidade das estimativas de incerteza, fundamentais para a detecção de erros de anotação. Ferramentas como o Cleanlab tendem a se beneficiar de bases de dados maiores e mais diversas, que permitam uma identificação mais precisa de ruídos nos rótulos.

Além disso, o desequilíbrio entre as classes representa uma dificuldade adicional. A classe NTL, por exemplo, possui apenas 26 amostras no conjunto de teste. Esse volume extremamente baixo dificulta tanto o aprendizado do modelo quanto a avaliação do desempenho, além de tornar a detecção de erros nessa classe particularmente desafiadora.

Outro fator relevante está relacionado à natureza das features extraídas. Neste trabalho, optou-se por utilizar descritores manuais baseados em histogramas de intensidade, uma abordagem simples, porém limitada para capturar a complexidade visual de imagens médicas. Essa limitação pode ter impactado negativamente a qualidade das predições do classificador, afetando, por consequência, a eficácia do Cleanlab na identificação de rótulos incorretos.

Portanto, os resultados indicam que, embora o uso de técnicas de supervisão fraca e detecção automática de erros seja promissor, sua aplicação em contextos médicos requer cautela. Para alcançar melhores resultados, seria recomendável o uso de features mais robustas, bases de dados mais extensas e balanceadas, além de uma análise humana criteriosa para validar quaisquer alterações nos rótulos.